## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-06-14-52-48 +0000,nypost,McDonald’s sales bounce back from drastic decl...,https://nypost.com/2025/08/06/business/mcdonal...
1,2025-08-06-14-52-31 +0000,nyt,Wildfire In Wet Washington State Is Changing U...,https://www.nytimes.com/2025/08/06/climate/was...
2,2025-08-06-14-50-35 +0000,nypost,Apple commits another $100B to US manufacturin...,https://nypost.com/2025/08/06/business/apple-c...
3,2025-08-06-14-49-27 +0000,bbc,Trump hits India with extra 25% tariff for buy...,https://www.bbc.com/news/articles/c1dxr1g4y7yo...
4,2025-08-06-14-47-34 +0000,nyt,Trump to Announce Additional $100 Billion Appl...,https://www.nytimes.com/2025/08/06/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
21,trump,55
258,will,14
270,new,12
22,tariffs,11
471,up,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
176,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...,115
97,2025-08-06-10-11-50 +0000,nypost,Trump envoy Steve Witkoff meets Putin ahead of...,https://nypost.com/2025/08/06/world-news/trump...,109
134,2025-08-06-09-00-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...,106
26,2025-08-06-14-04-00 +0000,wsj,Trump Envoy Witkoff Meets With Putin as Russia...,https://www.wsj.com/world/russia/trump-envoy-w...,105
284,2025-08-05-19-42-59 +0000,nypost,Zelensky says Ukraine ready to ink drone deal ...,https://nypost.com/2025/08/05/us-news/zelensky...,101


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
176,115,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...
108,59,2025-08-06-10-00-00 +0000,latimes,Chabria: Newsom vows Texas will be 'neutered' ...,https://www.latimes.com/california/story/2025-...
112,49,2025-08-06-10-00-00 +0000,nypost,NY POSTcast Daily Debrief: Death row inmate cr...,https://nypost.com/2025/08/06/us-news/ny-postc...
134,47,2025-08-06-09-00-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...
342,42,2025-08-05-16-34-00 +0000,wsj,The Swiss president is heading to the U.S. in ...,https://www.wsj.com/economy/trade/swiss-presid...
71,41,2025-08-06-11-49-45 +0000,nypost,Virgin Atlantic passenger threatens to rape fl...,https://nypost.com/2025/08/06/world-news/staff...
59,38,2025-08-06-12-45-15 +0000,nyt,"Alec Luhn, Missing American Journalist, Found ...",https://www.nytimes.com/2025/08/06/world/europ...
209,36,2025-08-05-23-19-49 +0000,latimes,L.A. County residents illegally exported 'sens...,https://www.latimes.com/california/story/2025-...
344,34,2025-08-05-16-15-00 +0000,wsj,The way Illinois draws political district boun...,https://www.wsj.com/politics/policy/pritzkers-...
233,34,2025-08-05-22-21-43 +0000,nypost,"Trump rips ‘no talent’ Gayle King, citing New ...",https://nypost.com/2025/08/05/media/trump-rips...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
